In [2]:
# import bibliotek

from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64
from PIL import Image
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

Using TensorFlow backend.


In [3]:
# zdefiniowanie funkcji send_control - przesyłanie parametrów sterowania do aplikacji

sio = socketio.Server()
app = Flask(__name__)

def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

In [4]:
# wczytanie wytrenowanego modelu
model = load_model('D:\\model.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 78, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 58, 78, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 76, 32)        18464     
_________________________________________________________________
activation_2 (Activation)    (None, 56, 76, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 38, 32)        0         
_______________________________________

In [6]:
# zdefiniowanie rozmiaru zdjeć, na taki na których uczony był model
def process_image(image):
    image=image[10:130:2,::4,:]
    return image

# utworzenie prostego kontrolera prędkości
class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.0
        self.error = 0.0
        self.integral = 0.0

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error

        return self.Kp * self.error + self.Ki * self.integral


# ustalenie parametrów początkowych dla kontrolera prędkości
controller = SimplePIController(0.1, 0.001)
set_speed = 7
controller.set_desired(set_speed)

# zdefiniowanie funkcji telemetry
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
            
        # bieżąca prędkość samochodu
        speed = float(data["speed"])
        
        # bieżące zdjecie z kamery środkowej
        imgString = data["image"]
        
        # konwersja zdekodowanego obrazu
        image = Image.open(BytesIO(base64.b64decode(imgString)))
        image_array = np.asarray(image)
        image_array = process_image(image_array)     
        img_batch = np.expand_dims(image_array, axis=0)
        
        # bieżący kąt sterowania 
        steering_angle = float(model.predict(img_batch))
        throttle = controller.update(float(speed))

        # przesłanie parametrów sterowania
        send_control(steering_angle, throttle)

    else:
        sio.emit('manual', data={}, skip_sid=True)

app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)


(3388) wsgi starting up on http://0.0.0.0:4567
(3388) accepted ('127.0.0.1', 53439)
wsgi exiting
127.0.0.1 - - [20/Jun/2019 22:32:10] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 116.781680
(3388) wsgi exited, is_accepting=True
